In [1]:
pip install torch transformers pandas openpyxl xlsxwriter


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 4.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
import torch
import numpy as np
from transformers import AutoModel, AutoTokenizer

# Load the CodeT5+ model and tokenizer
checkpoint = "Salesforce/codet5p-110m-embedding"
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)
model = AutoModel.from_pretrained(checkpoint, trust_remote_code=True).to(device)

# Load the Excel file from the specified path
df = pd.read_excel("/content/ML_DATASET.xlsx")
print("Data loaded successfully:")
print(df.head())  # Display the first few rows of the DataFrame

# Function to encode data in batches
def encode_data_batch(df, model, tokenizer, device, start_row, end_row):
    embeddings = []
    for i in range(start_row, end_row):
        code = df['Correct Solution'][i]  # Use the 'java' column
        inputs = tokenizer.encode(code, return_tensors="pt", max_length=512, truncation=True).to(device)
        with torch.no_grad():
            embedding = model(inputs)[0].cpu().numpy()  # Get fixed-size embeddings
        embeddings.append(embedding)
    return embeddings

# Process the data in batches
batch_size = 300
all_embeddings = []

start_row = 0
while start_row < len(df):
    end_row = min(start_row + batch_size, len(df))
    embeddings_batch = encode_data_batch(df, model, tokenizer, device, start_row, end_row)
    all_embeddings.extend(embeddings_batch)
    start_row = end_row

# Convert to numpy array and reshape if necessary
embeddings_array = np.vstack(all_embeddings)

# Create DataFrame from embeddings
embedding_columns = [f'python_t5_embedding_{i}' for i in range(embeddings_array.shape[1])]
embeddings_df = pd.DataFrame(embeddings_array, columns=embedding_columns)

# Save embeddings to Excel with a neatly formatted sheet
output_path = "/content/codeonlt_python_embeddings.xlsx"
with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
    embeddings_df.to_excel(writer, index=False, sheet_name="Embeddings")
    workbook = writer.book
    worksheet = writer.sheets["Embeddings"]

    # Set column widths for neat formatting
    for i, column in enumerate(embeddings_df.columns):
        max_len = max(embeddings_df[column].astype(str).map(len).max(), len(column))
        worksheet.set_column(i, i, max_len + 2)  # Adjust the padding as needed

print(f"Embeddings have been saved to {output_path}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/511k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

...

A new version of the following files was downloaded from https://huggingface.co/Salesforce/codet5p-110m-embedding:
- configuration_codet5p_embedding.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_codet5p_embedding.py:   0%|          | 0.00/1.93k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Salesforce/codet5p-110m-embedding:
- modeling_codet5p_embedding.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

Data loaded successfully:
                                    Correct Solution
0         def reverse_string(s):\n    return s[::-1]
1  def reverse_string(s):\n    reversed_str = ''\...
2  def reverse_string(s):\n    if len(s) == 0:\n ...
3  def reverse_string(s):\n    return ''.join(rev...
4  def reverse_string(s):\n    stack = list(s)\n ...
Embeddings have been saved to /content/codeonlt_python_embeddings.xlsx


In [7]:
# Step 1: Install required libraries
!pip install hdbscan umap-learn openpyxl

import pandas as pd
import umap

# Step 2: Load the embeddings
embeddings_df = pd.read_excel("/content/codeonlt_python_embeddings.xlsx")

# Step 3: Select the embedding data
# Adjust this if you want to select specific columns
embedding_data = embeddings_df.iloc[:, :].values  # Use all embedding columns

# Step 4: Apply UMAP for dimensionality reduction to 2 components
umap_model = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=4, random_state=42)
umap_embeddings = umap_model.fit_transform(embedding_data)

# Step 5: Create a DataFrame from UMAP embeddings
umap_df = pd.DataFrame(umap_embeddings, columns=['UMAP_1', 'UMAP_2','UMAP_3','UMAP_4'])

# Step 6: Save the UMAP results to a new Excel file
output_path = "/content/pythont5_umap4_embeddings.xlsx"
umap_df.to_excel(output_path, index=False)

print(f"UMAP embeddings have been saved to {output_path}")


/usr/local/lib/python3.10/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


UMAP embeddings have been saved to /content/pythont5_umap4_embeddings.xlsx


In [4]:
import pandas as pd
from sklearn.cluster import DBSCAN
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/pythont5_umap2_embeddings.xlsx"
df = pd.read_excel(file_path)

# Convert DataFrame to numpy array for DBSCAN
embeddings = df.values

# Apply DBSCAN clustering
# You can adjust eps based on your data characteristics
dbscan = DBSCAN(eps=0.5, min_samples=5)  # min_samples can be set to a low value
cluster_labels = dbscan.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 20


In [6]:
import pandas as pd
from sklearn.cluster import DBSCAN
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/pythont5_umap3_embeddings.xlsx"
df = pd.read_excel(file_path)

# Convert DataFrame to numpy array for DBSCAN
embeddings = df.values

# Apply DBSCAN clustering
# You can adjust eps based on your data characteristics
dbscan = DBSCAN(eps=0.5, min_samples=5)  # min_samples can be set to a low value
cluster_labels = dbscan.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 25


In [8]:
import pandas as pd
from sklearn.cluster import DBSCAN
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/pythont5_umap4_embeddings.xlsx"
df = pd.read_excel(file_path)

# Convert DataFrame to numpy array for DBSCAN
embeddings = df.values

# Apply DBSCAN clustering
# You can adjust eps based on your data characteristics
dbscan = DBSCAN(eps=0.5, min_samples=5)  # min_samples can be set to a low value
cluster_labels = dbscan.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 26


HERE ONWARDS AST


In [9]:
import pandas as pd
import torch
import numpy as np
from transformers import AutoModel, AutoTokenizer

# Load the CodeT5+ model and tokenizer
checkpoint = "Salesforce/codet5p-110m-embedding"
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)
model = AutoModel.from_pretrained(checkpoint, trust_remote_code=True).to(device)

# Load the Excel file from the specified path
df = pd.read_excel("/content/AST.xlsx")
print("Data loaded successfully:")
print(df.head())  # Display the first few rows of the DataFrame

# Function to encode data in batches
def encode_data_batch(df, model, tokenizer, device, start_row, end_row):
    embeddings = []
    for i in range(start_row, end_row):
        code = df['AST_PYTHON'][i]  # Use the 'java' column
        inputs = tokenizer.encode(code, return_tensors="pt", max_length=512, truncation=True).to(device)
        with torch.no_grad():
            embedding = model(inputs)[0].cpu().numpy()  # Get fixed-size embeddings
        embeddings.append(embedding)
    return embeddings

# Process the data in batches
batch_size = 300
all_embeddings = []

start_row = 0
while start_row < len(df):
    end_row = min(start_row + batch_size, len(df))
    embeddings_batch = encode_data_batch(df, model, tokenizer, device, start_row, end_row)
    all_embeddings.extend(embeddings_batch)
    start_row = end_row

# Convert to numpy array and reshape if necessary
embeddings_array = np.vstack(all_embeddings)

# Create DataFrame from embeddings
embedding_columns = [f'python_ast_t5_embedding_{i}' for i in range(embeddings_array.shape[1])]
embeddings_df = pd.DataFrame(embeddings_array, columns=embedding_columns)

# Save embeddings to Excel with a neatly formatted sheet
output_path = "/content/ast_python_embeddings.xlsx"
with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
    embeddings_df.to_excel(writer, index=False, sheet_name="Embeddings")
    workbook = writer.book
    worksheet = writer.sheets["Embeddings"]

    # Set column widths for neat formatting
    for i, column in enumerate(embeddings_df.columns):
        max_len = max(embeddings_df[column].astype(str).map(len).max(), len(column))
        worksheet.set_column(i, i, max_len + 2)  # Adjust the padding as needed

print(f"Embeddings have been saved to {output_path}")

Data loaded successfully:
                                          AST_PYTHON
0  (module (function_definition name: (identifier...
1  (module (function_definition name: (identifier...
2  (module (function_definition name: (identifier...
3  (module (function_definition name: (identifier...
4  (module (function_definition name: (identifier...
Embeddings have been saved to /content/ast_python_embeddings.xlsx


In [10]:
# Step 1: Install required libraries
!pip install hdbscan umap-learn openpyxl

import pandas as pd
import umap

# Step 2: Load the embeddings
embeddings_df = pd.read_excel("/content/ast_python_embeddings.xlsx")

# Step 3: Select the embedding data
# Adjust this if you want to select specific columns
embedding_data = embeddings_df.iloc[:, :].values  # Use all embedding columns

# Step 4: Apply UMAP for dimensionality reduction to 2 components
umap_model = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2, random_state=42)
umap_embeddings = umap_model.fit_transform(embedding_data)

# Step 5: Create a DataFrame from UMAP embeddings
umap_df = pd.DataFrame(umap_embeddings, columns=['UMAP_1', 'UMAP_2'])

# Step 6: Save the UMAP results to a new Excel file
output_path = "/content/python_ast_umap_embeddings.xlsx"
umap_df.to_excel(output_path, index=False)

print(f"UMAP embeddings have been saved to {output_path}")


/usr/local/lib/python3.10/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


UMAP embeddings have been saved to /content/python_ast_umap_embeddings.xlsx


In [12]:
import pandas as pd
from sklearn.cluster import DBSCAN
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/python_ast_umap_embeddings.xlsx"
df = pd.read_excel(file_path)

# Convert DataFrame to numpy array for DBSCAN
embeddings = df.values

# Apply DBSCAN clustering
# You can adjust eps based on your data characteristics
dbscan = DBSCAN(eps=0.5, min_samples=5)  # min_samples can be set to a low value
cluster_labels = dbscan.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 8


In [17]:
print("hi")

hi
